<a href="https://colab.research.google.com/github/iz-120/ECM514-Projeto02-Assistente-Diagnostico/blob/main/Projeto_2_Assistente_de_Diagn%C3%B3stico_de_Dengue_Pre_Processamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inicialização

## Instalações

* PySUS (Docs: https://pysus.readthedocs.io/en/latest/databases/SINAN.html)

In [1]:
!pip install PySUS --quiet

## Importações

In [2]:
import pandas as pd
import numpy as np
from pysus import SINAN
from pysus.preprocessing.decoders import decodifica_idade_SINAN
import time
from sklearn.preprocessing import OneHotEncoder
from google.colab import drive
from os import path

# Exibe todas as colunas
pd.set_option('display.max_columns', None)

## Funções

In [3]:
# ==============================================================================
# LIMPEZA DO DATAFRAME ANUAL
# ==============================================================================
def limpa_df(df):
    """
    Realiza a limpeza e pré-processamento de um dataframe anual do SINAN Dengue.

    Args:
        df: Arquivo carregado pela pysus após conversão para dataframe.

    Returns:
        dataframe limpo, processado e com tamanho reduzido.
    """
    print(f"    - Processando {len(df):,} linhas...")

    # --- 2.1. Seleção das colunas de interesse ---
    colunas_sintomas = [
        'FEBRE', 'MIALGIA', 'CEFALEIA', 'EXANTEMA', 'VOMITO',
        'NAUSEA', 'PETEQUIA_N', 'ARTRALGIA', 'DOR_RETRO', 'SANGRAM'
    ]
    colunas_demograficas = ['NU_IDADE_N', 'CS_SEXO', 'CS_RACA']
    colunas_chave = ['CLASSI_FIN', 'DT_NOTIFIC', 'DT_SIN_PRI', 'ID_AGRAVO']

    colunas_para_manter = colunas_chave + colunas_demograficas + colunas_sintomas
    df = df[colunas_para_manter].copy()

    # --- 2.2. Conversão de tipos das variáveis ---
    # Inicializa a coluna IDADE_ANOS
    df['IDADE_ANOS'] = -1 # Valor padrão para indicar que a idade ainda não foi calculada

    # Tenta decodificar idade usando decodifica_idade_SINAN, com fallback para cálculo manual
    def calcula_idade_segura(row):
        try:
            # Tenta decodificar usando a função PySUS
            return decodifica_idade_SINAN(row['NU_IDADE_N'], 'Y')
        except:
            # Se der erro, calcula manualmente usando DT_NOTIFIC e DT_NASC
            try:
              dt_notific = pd.to_datetime(row['DT_NOTIFIC'], errors='coerce')
              dt_nasc = pd.to_datetime(row['DT_NASC'], errors='coerce')
              if pd.notna(dt_notific) and pd.notna(dt_nasc):
                  return int((dt_notific - dt_nasc).days / 365.25)
              else:
                  return -9 # Retorna -9 se as datas forem inválidas
            except:
                return -99 # Retorna -99 se o cálculo manual falhar também

    df['IDADE_ANOS'] = df.apply(calcula_idade_segura, axis=1)

    # String
    colunas_string = ['ID_AGRAVO', 'CS_SEXO']
    for col in colunas_string:
      df[col] = df[col].astype(str)

    # Numérico
    colunas_num = ['CLASSI_FIN', 'CS_RACA', 'IDADE_ANOS'] + colunas_sintomas
    for col in colunas_num:
      df[col] = df[col].replace('', np.nan)
      df[col] = df[col].astype(float)

    # Data
    colunas_data = ['DT_SIN_PRI', 'DT_NOTIFIC']
    for col in colunas_data:
      df[col] = pd.to_datetime(df[col], errors='coerce')

    # Número de dias entre primeiros sintomas e notificação
    df['DIAS_COM_SINTOMAS'] = (df['DT_NOTIFIC'] - df['DT_SIN_PRI']).dt.days

    # --- 2.3. Filtragem de casos válidos ---
    # Filtrar apenas classificações finais confirmadas de Dengue
    codigos_confirmados = [10, 11, 12] # 10: Dengue, 11: Dengue c/ Sinais de Alarme, 12: Dengue Grave
    df = df[df['CLASSI_FIN'].isin(codigos_confirmados)]

    # Verifica se Dataframe está vazio após filtragem
    if df.empty:
        print("    - Dataframe vazio após filtragem. Retornando dataframe vazio.")
        return pd.DataFrame()

    # --- 2.4. Engenharia de Features ---
    # Criar variável alvo: 1 para casos graves (11 ou 12), 0 para não graves (10)
    df['RISCO_GRAVIDADE'] = np.where(df['CLASSI_FIN'].isin([11, 12]), 'grave', 'nao_grave')

    # Remove idades absurdas
    df = df[(df['IDADE_ANOS'] >= 0) & (df['IDADE_ANOS'] <= 120)]
    df['IDADE_ANOS'] = df['IDADE_ANOS'].astype(int)

    # Remove sexo vazio
    df = df.dropna(subset=['CS_SEXO'])
    df = df.drop(df[df['CS_SEXO'] == 9].index)

    # Remove raça vazia
    df = df.dropna(subset=['CS_RACA'])

    # Converte raça de numérico para categórico
    raca_map = {
        1.0: 'Branca',
        2.0: 'Parda',
        3.0: 'Preta',
        4.0: 'Amarela',
        5.0: 'Indigena',
        9.0: 'Ignorado'
    }
    df['CS_RACA'] = df['CS_RACA'].map(raca_map)


    # --- 2.5. Limpeza e Padronização ---
    # Converter colunas de sintomas para binário ('sim' para 'Sim', 'nao' para o resto)
    for col in colunas_sintomas:
        if col in df.columns:
            # Assume-se que '1' é Sim. NaN, '2' (Não), '9' (Ignorado) viram não.
            df[col] = np.where(df[col] == 1.0, 'sim', 'nao')

    # --- 2.6. One Hot Encoding ---
    # Seleciona colunas cetegoricas
    cols_categoricas = ['RISCO_GRAVIDADE', 'CS_SEXO', 'CS_RACA'] + colunas_sintomas

    # Inicializa encoder
    encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False) # instancia o estimador

    # Faz fit e transform
    encoded_data = encoder.fit_transform(df[cols_categoricas])

    # Cria dataframe com dados após encoding
    encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(cols_categoricas))

    # Concatena com o original excluindo colunas originais
    df = pd.concat([df.drop(columns=cols_categoricas), encoded_df], axis=1)

    # Remove uma coluna de cada feature para evitar redundância
    drop = ['RISCO_GRAVIDADE_nao_grave', 'CS_SEXO_M',
            'CS_RACA_Ignorado', 'FEBRE_nao', 'MIALGIA_nao',
            'CEFALEIA_nao', 'EXANTEMA_nao', 'VOMITO_nao',
            'NAUSEA_nao', 'PETEQUIA_N_nao', 'ARTRALGIA_nao',
            'DOR_RETRO_nao', 'SANGRAM_nao']
    df = df.drop(columns=drop)

    # --- 2.6. Finalização ---
    # Remover colunas originais que não serão mais usadas
    colunas_para_remover = ['CLASSI_FIN', 'DT_SIN_PRI', 'DT_NASC', 'ID_AGRAVO', 'NU_IDADE_N']
    df.drop(columns=colunas_para_remover, inplace=True, errors='ignore')

    print(f"    - Processamento concluído. {len(df):,} linhas válidas restantes.")
    return df

# Fonte de Dados

O conjunto de dados para este estudo será extraído do Sistema de Informação de Agravos de Notificação (SINAN), a base de dados oficial para o registro de doenças de notificação compulsória no Brasil.

O acesso aos microdados será realizado de forma programática utilizando a biblioteca de código aberto pysus, uma ferramenta desenvolvida para facilitar a aquisição e o pré-processamento de dados dos sistemas de informação do Sistema Único de Saúde (SUS)

In [8]:
# Carregando arquivos do DATASUS
sinan = SINAN().load()

# Carregamento dos Dados

**Obs.:** Este processo pode levar vários minutos

In [9]:
# ==============================================================================
# CARREGAMENTO DOS DADOS
# ==============================================================================
print("Iniciando o carregamento dos dados de Dengue (SINAN)...")
print("Este processo pode levar vários minutos, dependendo da sua conexão.")

start_time = time.time()

# Define o ano de interesse
ano = 2025

print(f"\nBuscando dados para o ano de {ano}...")
try:
    # A função 'sinan.get_files' busca os dados do agravo 'DENG' (Dengue) para o ano especificado.
    df_anual = sinan.get_files(dis_code="DENG", year=ano)[0].download().to_dataframe()

    # Verifica se o dataframe foi carregado corretamente e não está vazio
    if df_anual is not None and not len(df_anual)==0:
        print(f"-> Sucesso! {len(df_anual):,} linhas carregadas para {ano}.")
        df_limpo = limpa_df(df_anual)
    else:
        print(f"-> Aviso: Não foram encontrados dados para o ano {ano}.")

except Exception as e:
    # Captura de erro caso a internet falhe ou o arquivo não seja encontrado
    print(f"-> Erro ao carregar os dados de {ano}. Motivo: {e}")


# ==============================================================================
# VERIFICAÇÃO DO RESULTADO
# ==============================================================================
end_time = time.time()
execution_time = (end_time - start_time) / 60

print(f"Tempo total de execução: {execution_time:.2f} minutos.")

print(f"Amostra das 10 primeiras linhas do dataframe:")
display(df_limpo.head(10))



Iniciando o carregamento dos dados de Dengue (SINAN)...
Este processo pode levar vários minutos, dependendo da sua conexão.

Buscando dados para o ano de 2025...
-> Sucesso! 1,560,606 linhas carregadas para 2025.
    - Processando 1,560,606 linhas...
    - Processamento concluído. 1,516,068 linhas válidas restantes.
Tempo total de execução: 2.95 minutos.
Amostra das 10 primeiras linhas do dataframe:


,DT_NOTIFIC,IDADE_ANOS,DIAS_COM_SINTOMAS,RISCO_GRAVIDADE_grave,CS_SEXO_F,CS_SEXO_I,CS_RACA_Amarela,CS_RACA_Branca,CS_RACA_Indigena,CS_RACA_Parda,CS_RACA_Preta,FEBRE_sim,MIALGIA_sim,CEFALEIA_sim,EXANTEMA_sim,VOMITO_sim,NAUSEA_sim,PETEQUIA_N_sim,ARTRALGIA_sim,DOR_RETRO_sim
0,2025-04-22,33.0,6.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,2025-04-15,62.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2025-04-03,24.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
3,2025-04-15,7.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,2025-04-30,19.0,3.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,2025-04-16,32.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
6,2025-04-25,33.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
7,2025-04-16,54.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
8,2025-04-11,34.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
9,2025-04-24,56.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


# Download dos dados

Vale notar que essa etapa foi usada apenas uma vez para tornar o processo de importação dos dados mais simples no futuro ao remover a necessidade de acessar o pySUS repetidas vezes para fazer o download dos dados.

A célula está comentada, pois irá gerar erro em qualquer Drive que não possua o path indicado. Ela foi mantida apenas para registro do processo de desenvolvimento do projeto.

In [ ]:
# # Abrindo Drive
# drive.mount('/content/drive')

# # Definindo o path
# path = '/content/drive/MyDrive/Mauá/5° Ano/ECM514 - Ciência de Dados/Projeto 2/Dados'

# # Salva dataframe original em csv no Drive
# arquivo_original = path + '/dengue_2025.csv'
# df_anual.to_csv(arquivo_original, index=False)

# # Salva dataframe limpo em csv no Drive
# arquivo_limpo = path + '/dengue_limpo_2025.csv'
# df_limpo.to_csv(arquivo_limpo, index=False)